In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 87.8 MB/s eta 0:00:00


In [3]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

### Get text files from drive folder

In [6]:
import os

In [30]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define the path to the folder containing your text files on Google Drive
folder_path = '/content/drive/MyDrive/NLP analysis'

# Define the desired order of labels
label_order = {
    'Book 1 - The Philosopher\'s Stone.txt': 1,
    'Book 2 - The Chamber of Secrets.txt': 2,
    'Book 3 - The Prisoner of Azkaban.txt': 3,
    'Book 4 - The Goblet of Fire.txt': 4,
    'Book 5 - The Order of the Phoenix.txt': 5,
    'Book 6 - The Half Blood Prince.txt': 6,
    'Book 7 - The Deathly Hallows.txt': 7
}

# Prepare the training data by reading the text files
texts = []
labels = []

for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    if os.path.isfile(file_path) and file_name.endswith('.txt'):
        with open(file_path, 'r') as f:
            text = f.read()
            texts.append(text)
            label = label_order.get(file_name)
            if label is not None:
                labels.append(label)

                print(f"File: {file_name}, Label: {label}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
File: Book 4 - The Goblet of Fire.txt, Label: 4
File: Book 2 - The Chamber of Secrets.txt, Label: 2
File: Book 7 - The Deathly Hallows.txt, Label: 7
File: Book 6 - The Half Blood Prince.txt, Label: 6
File: Book 5 - The Order of the Phoenix.txt, Label: 5
File: Book 3 - The Prisoner of Azkaban.txt, Label: 3
File: Book 1 - The Philosopher's Stone.txt, Label: 1


Some files are missing.
['B', 'o', 'o', 'k', ' ', '1', ' ', '-', ' ', 'T', 'h', 'e', ' ', 'P', 'h', 'i', 'l', 'o', 's', 'o', 'p', 'h', 'e', 'r', "'", 's', ' ', 'S', 't', 'o', 'n', 'e', '.', 't', 'x', 't']


B
o
o
k
 
1
 
-
 
T
h
e
 
P
h
i
l
o
s
o
p
h
e
r
'
s
 
S
t
o
n
e
.
t
x
t


In [ ]:
for label, file_name in zip(labels, file_names):
    print(f"Label: {label}, File Name: {file_name}")